# Введение в MapReduce модель на Python


In [65]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [66]:
def MAP(_, row:NamedTuple):#фильтрует только женщин и создаёт генератор по возрасту
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):#вычисляет среднее кол-во социальных контактов для каждого возраста
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [67]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [68]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [69]:
def RECORDREADER():#возвращает список из кортежей
  return [(u.id, u) for u in input_collection]

In [70]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [71]:
def flatten(nested_iterable):#превращает в один поток элементов
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [72]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [73]:
def groupbykey(iterable):#создаёт список по каждому ключу
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [74]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [75]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [76]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных. 

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [77]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL 

In [78]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str
    
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication 

In [79]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])
 
def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])
      
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, np.float64(1.992737721273661)),
 (1, np.float64(1.992737721273661)),
 (2, np.float64(1.992737721273661)),
 (3, np.float64(1.992737721273661)),
 (4, np.float64(1.992737721273661))]

## Inverted index 

In [80]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)
      
def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)
 
def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [81]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [82]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()
      
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]
 
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers
  
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  
  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*
 
flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount 

In [83]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)
      
  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)
  
# try to set COMBINER=REDUCER and look at the number of values sent over the network 
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2)]),
 (1, [('is', 18), ('it', 18), ('what', 10)])]

## TeraSort

In [84]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for value in split:
        yield (value, None)
      
  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])
    
def MAP(value:int, _):
  yield (value, None)
  
def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)
  
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, np.float64(0.06920559836052276)),
   (None, np.float64(0.12173216900865103)),
   (None, np.float64(0.1817042128829388)),
   (None, np.float64(0.19325109668025076)),
   (None, np.float64(0.321116136827419)),
   (None, np.float64(0.3337349779353528)),
   (None, np.float64(0.33996819941232126)),
   (None, np.float64(0.3586860630572992)),
   (None, np.float64(0.3788337414780564)),
   (None, np.float64(0.4361854996078356)),
   (None, np.float64(0.4451172957402918)),
   (None, np.float64(0.4509811711198375)),
   (None, np.float64(0.46371469226215334)),
   (None, np.float64(0.4742749393390585)),
   (None, np.float64(0.4851347551574403))]),
 (1,
  [(None, np.float64(0.5047026433140964)),
   (None, np.float64(0.524412698043214)),
   (None, np.float64(0.5604990166947471)),
   (None, np.float64(0.5643477871275202)),
   (None, np.float64(0.5858708108576638)),
   (None, np.float64(0.6130182552059563)),
   (None, np.float64(0.6248498659322991)),
   (None, np.float64(0.717474501024806)

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [85]:
import csv
from typing import NamedTuple
import pandas as pd

class Station(NamedTuple):
    id: int
    name: str
    lat: float
    long: float
    dock_count: int
    city: str
    installation_date: str

df = pd.read_csv("C:/Users/L/Downloads/stations.csv")

input_collection = [
    Station(row.id, row.name, row.lat, row.long, row.dock_count, row.city, row.installation_date)
    for row in df.itertuples(index=False)
]

In [86]:
# Ищем станцию с максимальным количеством парковочных мест
def RECORDREADER():
    return [(station.id, station) for station in input_collection]

def MAP_MAX(_, row:NamedTuple):
    yield (0, row)

def REDUCE_MAX(_, rows:Iterator[NamedTuple]):
    max_station = 0
    station = None
    for row in rows:
        if row.dock_count > max_station:
            max_station = row.dock_count
            station = row
    yield station

output = MapReduce(RECORDREADER, MAP_MAX, REDUCE_MAX)
output = list(output)
output

[Station(id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.901782, dock_count=27, city='San Jose', installation_date='8/6/2013')]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [87]:
# Ищем арифтметическое среднее для парковочных мест на станции
def RECORDREADER():
    return [(station.id, station) for station in input_collection]

def MAP_AVG(_, row:NamedTuple):
    yield (0, row)

def REDUCE_AVG(_, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.dock_count
    count += 1
  if (count > 0):
    yield sum/count
  else:
    yield 0
      
output = MapReduce(RECORDREADER, MAP_AVG, REDUCE_AVG)
output = list(output)
output

[17.65714285714286]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [88]:
# Ищем в каждом городе станцию с максимальным количеством парковочных мест
def RECORDREADER():
    return [(station.id, station) for station in input_collection]

def MAP_MAX(_, row:NamedTuple):
    yield (row.city, row)

def REDUCE_MAX(_, rows:Iterator[NamedTuple]):
    max_station = 0
    station = None
    for row in rows:
        if row.dock_count > max_station:
            max_station = row.dock_count
            station = row
    yield station
    
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

output = MapReduce(RECORDREADER, MAP_MAX, REDUCE_MAX)
output = list(output)
output


[Station(id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.901782, dock_count=27, city='San Jose', installation_date='8/6/2013'),
 Station(id=22, name='Redwood City Caltrain Station', lat=37.486078000000006, long=-122.232089, dock_count=25, city='Redwood City', installation_date='8/15/2013'),
 Station(id=28, name='Mountain View Caltrain Station', lat=37.394358, long=-122.07671299999998, dock_count=23, city='Mountain View', installation_date='8/15/2013'),
 Station(id=34, name='Palo Alto Caltrain Station', lat=37.443988, long=-122.164759, dock_count=23, city='Palo Alto', installation_date='8/14/2013'),
 Station(id=61, name='2nd at Townsend', lat=37.780526, long=-122.39028799999998, dock_count=27, city='San Francisco', installation_date='8/22/2013')]

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [89]:
# Добавим дубликаты чтобы проверить функцию
input_collection += input_collection[:2]
input_collection += [input_collection[0]]

maps = 2
reducers = 2

def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for station in split:
            yield (station.id, station)

    split_size = int(np.ceil(len(input_collection) / maps))
    for i in range(0, len(input_collection), split_size):
        yield RECORDREADER(input_collection[i:i + split_size])

def MAP(_, station: Station):
    yield (station, None)

def REDUCE(station: Station, _):
    yield station
    
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
unique_count = sum(len(partition) for _, partition in partitioned_output)
print(f" Уникальных станций после удаления дубликатов: {unique_count}")

73 key-value pairs were sent over a network.
 Уникальных станций после удаления дубликатов: 70


### Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [90]:
# Станции только в городе Сан-Франциско
def RECORDREADER():
    return [(station.id, station) for station in input_collection]

def MAP_SELECT(_, row:NamedTuple):
    if row.city == "San Francisco":
        yield (row.id, row)

def REDUCE_SELECT(id:int, rows:Iterator[NamedTuple]):
    yield (id, rows)

output = MapReduce(RECORDREADER, MAP_SELECT, REDUCE_SELECT)
output = list(output)
output

[(41,
  [Station(id=41, name='Clay at Battery', lat=37.795001, long=-122.39997, dock_count=15, city='San Francisco', installation_date='8/19/2013')]),
 (42,
  [Station(id=42, name='Davis at Jackson', lat=37.79728, long=-122.398436, dock_count=15, city='San Francisco', installation_date='8/19/2013')]),
 (45,
  [Station(id=45, name='Commercial at Montgomery', lat=37.794231, long=-122.402923, dock_count=15, city='San Francisco', installation_date='8/19/2013')]),
 (46,
  [Station(id=46, name='Washington at Kearney', lat=37.795425, long=-122.404767, dock_count=15, city='San Francisco', installation_date='8/19/2013')]),
 (47,
  [Station(id=47, name='Post at Kearney', lat=37.788975, long=-122.403452, dock_count=19, city='San Francisco', installation_date='8/19/2013')]),
 (48,
  [Station(id=48, name='Embarcadero at Vallejo', lat=37.799953, long=-122.398525, dock_count=15, city='San Francisco', installation_date='8/19/2013')]),
 (49,
  [Station(id=49, name='Spear at Folsom', lat=37.790302, long

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [91]:
S = ["name", "city"]

def RECORDREADER():
    return [(station.id, station) for station in input_collection]
    
def MAP_PROJECT(_, row:NamedTuple):
    t_proj = tuple(getattr(row, field) for field in S)
    yield (t_proj, row)

def REDUCE_PROJECT(t_proj, rows: Iterator[NamedTuple]):
    yield (t_proj, rows[0].id)

output = MapReduce(RECORDREADER, MAP_PROJECT, REDUCE_PROJECT)
output = list(output)
output

[(('San Jose Diridon Caltrain Station', 'San Jose'), 2),
 (('San Jose Civic Center', 'San Jose'), 3),
 (('Santa Clara at Almaden', 'San Jose'), 4),
 (('Adobe on Almaden', 'San Jose'), 5),
 (('San Pedro Square', 'San Jose'), 6),
 (('Paseo de San Antonio', 'San Jose'), 7),
 (('San Salvador at 1st', 'San Jose'), 8),
 (('Japantown', 'San Jose'), 9),
 (('San Jose City Hall', 'San Jose'), 10),
 (('MLK Library', 'San Jose'), 11),
 (('SJSU 4th at San Carlos', 'San Jose'), 12),
 (('St James Park', 'San Jose'), 13),
 (('Arena Green / SAP Center', 'San Jose'), 14),
 (('SJSU - San Salvador at 9th', 'San Jose'), 16),
 (('Franklin at Maple', 'Redwood City'), 21),
 (('Redwood City Caltrain Station', 'Redwood City'), 22),
 (('San Mateo County Center', 'Redwood City'), 23),
 (('Redwood City Public Library', 'Redwood City'), 24),
 (('Stanford in Redwood City', 'Redwood City'), 25),
 (('Redwood City Medical Center', 'Redwood City'), 26),
 (('Mountain View City Hall', 'Mountain View'), 27),
 (('Mountain V

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [92]:
# Находим объединение двух пересекающихся датасетов
dataset1 = input_collection[:5]
dataset2 = input_collection[3:8]

def RECORDREADER():
    return [(station.id, station) for station in dataset1] + [(station.id, station) for station in dataset2]

def MAP_UNION(_, row: NamedTuple):
    yield (row.id, row)

def REDUCE_UNION(id: int, rows: Iterator[NamedTuple]):
    yield (id, rows)

output = MapReduce(RECORDREADER, MAP_UNION, REDUCE_UNION)
output = list(output)
output

[(2,
  [Station(id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.901782, dock_count=27, city='San Jose', installation_date='8/6/2013')]),
 (3,
  [Station(id=3, name='San Jose Civic Center', lat=37.330698, long=-121.888979, dock_count=15, city='San Jose', installation_date='8/5/2013')]),
 (4,
  [Station(id=4, name='Santa Clara at Almaden', lat=37.333988, long=-121.894902, dock_count=11, city='San Jose', installation_date='8/6/2013')]),
 (5,
  [Station(id=5, name='Adobe on Almaden', lat=37.331415, long=-121.8932, dock_count=19, city='San Jose', installation_date='8/5/2013'),
   Station(id=5, name='Adobe on Almaden', lat=37.331415, long=-121.8932, dock_count=19, city='San Jose', installation_date='8/5/2013')]),
 (6,
  [Station(id=6, name='San Pedro Square', lat=37.336721, long=-121.894074, dock_count=15, city='San Jose', installation_date='8/7/2013'),
   Station(id=6, name='San Pedro Square', lat=37.336721, long=-121.894074, dock_count=15, city='San Jose', installa

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [93]:
# Находим пересечение двух пересекающихся датасетов
dataset1 = input_collection[:5]
dataset2 = input_collection[3:8]

def RECORDREADER():
    return [(station.id, station) for station in dataset1] + [(station.id, station) for station in dataset2]

def MAP_INTERSECTION(_, row: NamedTuple):
    yield (row.id, row)

def REDUCE_INTERSECTION(id: int, rows: Iterator[NamedTuple]):
    if len(rows) == 2:
        yield (id, rows)

output = MapReduce(RECORDREADER, MAP_INTERSECTION, REDUCE_INTERSECTION)
output = list(output)
output

[(5,
  [Station(id=5, name='Adobe on Almaden', lat=37.331415, long=-121.8932, dock_count=19, city='San Jose', installation_date='8/5/2013'),
   Station(id=5, name='Adobe on Almaden', lat=37.331415, long=-121.8932, dock_count=19, city='San Jose', installation_date='8/5/2013')]),
 (6,
  [Station(id=6, name='San Pedro Square', lat=37.336721, long=-121.894074, dock_count=15, city='San Jose', installation_date='8/7/2013'),
   Station(id=6, name='San Pedro Square', lat=37.336721, long=-121.894074, dock_count=15, city='San Jose', installation_date='8/7/2013')])]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [94]:
# Находим разницу двух пересекающихся датасетов
dataset1 = input_collection[:5]
dataset2 = input_collection[3:8]

def RECORDREADER():
    return [(1, station) for station in dataset1] + [(2, station) for station in dataset2]

def MAP_DIFFERENCE(id:int, row:NamedTuple):
    yield (row, id)

def REDUCE_DIFFERENCE(rows: Iterator[NamedTuple], ids: int):
    if ids == [2]:
        yield rows

output = MapReduce(RECORDREADER, MAP_DIFFERENCE, REDUCE_DIFFERENCE)
output = list(output)
output

[Station(id=7, name='Paseo de San Antonio', lat=37.333798, long=-121.886943, dock_count=15, city='San Jose', installation_date='8/7/2013'),
 Station(id=8, name='San Salvador at 1st', lat=37.330165, long=-121.885831, dock_count=15, city='San Jose', installation_date='8/5/2013'),
 Station(id=9, name='Japantown', lat=37.348742, long=-121.894715, dock_count=15, city='San Jose', installation_date='8/5/2013')]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [95]:
class City(NamedTuple):
  city: str
  region: str
    
city_collection = [
    City(city="San Jose", region="South Bay"),
    City(city="San Francisco", region="North Bay"),
    City(city="Mountain View", region="Mid Peninsula"),
    City(city="Palo Alto", region="Mid Peninsula"),
    City(city="Redwood City", region="Mid Peninsula")
]

def RECORDREADER():
    return [(station.city, station) for station in input_collection] + [(city.city, city) for city in city_collection]

def MAP_NJOIN(city, row:NamedTuple):
    yield (city, row)

def REDUCE_NJOIN(city, rows:Iterator[NamedTuple]):
    city = None
    for row in rows:
        if type(row) is City:
            city = row

    for row in rows:
        if type(row) is Station:
            yield (row, row.city, city)


output = MapReduce(RECORDREADER, MAP_NJOIN, REDUCE_NJOIN)
output = list(output)
njoin = output
output

[(Station(id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.901782, dock_count=27, city='San Jose', installation_date='8/6/2013'),
  'San Jose',
  City(city='San Jose', region='South Bay')),
 (Station(id=3, name='San Jose Civic Center', lat=37.330698, long=-121.888979, dock_count=15, city='San Jose', installation_date='8/5/2013'),
  'San Jose',
  City(city='San Jose', region='South Bay')),
 (Station(id=4, name='Santa Clara at Almaden', lat=37.333988, long=-121.894902, dock_count=11, city='San Jose', installation_date='8/6/2013'),
  'San Jose',
  City(city='San Jose', region='South Bay')),
 (Station(id=5, name='Adobe on Almaden', lat=37.331415, long=-121.8932, dock_count=19, city='San Jose', installation_date='8/5/2013'),
  'San Jose',
  City(city='San Jose', region='South Bay')),
 (Station(id=6, name='San Pedro Square', lat=37.336721, long=-121.894074, dock_count=15, city='San Jose', installation_date='8/7/2013'),
  'San Jose',
  City(city='San Jose', region='Sou

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [96]:
# Находим среднее число парковочных мест по региону
def RECORDREADER():
    return njoin

def MAP_GROUP(station: Station, city_name: str, city: City):
    yield (city.region, station.dock_count)

def REDUCE_GROUP(region, counts: Iterator[int]):
    counts = list(counts)
    avg = sum(counts) / len(counts) if counts else 0
    yield (region, avg)

output = MapReduce(RECORDREADER, MAP_GROUP, REDUCE_GROUP)
output = list(output)
output

[('South Bay', 17.526315789473685),
 ('Mid Peninsula', 16.157894736842106),
 ('North Bay', 19.0)]

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [97]:
mat = np.array([[1, 2, 3, 4, 5, 6],
                [7, 8, 9, 10, 11, 12],
                [13, 14, 15, 16, 17, 18]])


vec = np.array([1, 2, 3, 4, 5, 6])

stripe_width = 2
stripes_num = int(len(mat[0])/stripe_width)

def RECORDREADER():
  a = 0
  b = stripe_width
  for s in range(stripes_num):
    stripe_mat = mat[:,a:b]
    stripe_vec = vec[a:b]
    a += stripe_width
    b += stripe_width
    yield (s ,(stripe_mat, stripe_vec))

def MAP(num, stripe):
  yield (num, stripe[0] @ stripe[1])

def REDUCE(num, results):
  yield (num, results)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, [array([ 5, 23, 41])]),
 (1, [array([ 25,  67, 109])]),
 (2, [array([ 61, 127, 193])])]

In [98]:
result = np.sum([v[1][0] for v in output], axis=0)
result

array([ 91, 217, 343])

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$. 





In [99]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [100]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])
      
def MAP(k1, v1):
  (j, k) = k1
  w = v1
  # solution code that yield(k2,v2) pairs
  for i in range(I):
    k2 = (i, k)
    v2 = small_mat[i, j] * w
    yield (k2, v2)

def REDUCE(key, values):
  (i, k) = key
  # solution code that yield(k3,v3) pairs
  k3 = (i, k)
  v3 = 0
  for j in range(J):
    v3 += values[j]
  yield (k3, v3)

Проверьте своё решение

In [101]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [102]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [103]:
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for i in range(I):
    for j in range(J):
      for k in range(K):
        yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))

def MAP(kv1, kv2):
  (i, j), v1 = kv1
  (j, k), v2 = kv2
  yield ((i, k), v1*v2)

def REDUCE(key, values):
  v3 = 0
  for j in range(J):
    v3 += values[j]
  yield (key, v3)

In [104]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER. 

In [105]:
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for i in split:
      for j in range(J):
        for k in range(K):
          yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))

  split_size = int(np.ceil(I/maps))
  for i in range(0, I, split_size):
    yield RECORDREADER(range(i, i+split_size))

def MAP(kv1, kv2):
  (i, j), v1 = kv1
  (j, k), v2 = kv2
  yield ((i, k), v1*v2)

def REDUCE(key, values):
  v3 = 0
  for j in range(J):
    v3 += values[j]
  yield (key, v3)

In [106]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

Да, решение будет работать, даже если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы, но может быть неполным или содержать ошибки из-за дублирования. 